In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [28]:
train = pd.read_csv('C:/Users/s_csmscox/jupyterSave/fashion-mnist_train.csv')
test = pd.read_csv('C:/Users/s_csmscox/jupyterSave/fashion-mnist_test.csv')

train_label = train['label']
train.drop(['label'], axis=1, inplace=True)

test_label = test['label']
test.drop(['label'], axis=1, inplace=True)

# 정규화
scaler_train = MinMaxScaler()
scaler_train.fit(train)
x_data_train_norm = scaler_train.transform(train).astype(np.float32)

scaler_test = MinMaxScaler()
scaler_test.fit(test)
x_data_test_norm = scaler_test.transform(test).astype(np.float32)

# one hot encoding
### tensorflow 기능을 이용해서 one hot encoding을 생성
sess = tf.Session()
t_data_train_onehot = sess.run(tf.one_hot(train_label, depth=10))


# reshape
x_data_train_norm = x_data_train_norm.reshape(60000,28,28,1)
x_data_test_norm = x_data_test_norm.reshape(10000,28,28,1)

In [35]:
# placeholder
X = tf.placeholder(shape=[None,28,28,1], dtype=tf.float32)
T = tf.placeholder(shape=[None,10], dtype=tf.float32)

# Weight & bias => filter(kernel)
W1 = tf.Variable(tf.random.normal([3,3,1,32]))

conv1 = tf.nn.conv2d(X,
                      W1,
                      strides=[1,1,1,1],
                      padding='SAME')
relu1 = tf.nn.relu(conv1)
layer1 = tf.nn.max_pool(relu1,
                         ksize=[1,2,2,1],
                         strides=[1,2,2,1],
                         padding='VALID') # (?, 14, 14, 32)

W2 = tf.Variable(tf.random.normal([3,3,32,64]))
conv2 = tf.nn.conv2d(layer1,
                      W2,
                      strides=[1,1,1,1],
                      padding='SAME')
relu2 = tf.nn.relu(conv2)
_layer2 = tf.nn.max_pool(relu2,
                         ksize=[1,2,2,1],
                         strides=[1,1,1,1],
                         padding='VALID') # (?, 7, 7, 64)
layer2 = tf.nn.dropout(_layer2, 0.75)

W3 = tf.Variable(tf.random.normal([3,3,64,128]))
conv3 = tf.nn.conv2d(layer2,
                      W3,
                      strides=[1,1,1,1],
                      padding='SAME')
relu3 = tf.nn.relu(conv3)
layer3 = tf.nn.max_pool(relu3,
                         ksize=[1,2,2,1],
                         strides=[1,1,1,1],
                         padding='VALID') # (?, 3, 3, 128)

flat = tf.reshape(layer3, [-1,4*4*128])

W4 = tf.Variable(tf.random.normal([4*4*128, 10]))
logit = tf.matmul(flat, W4)
H = tf.nn.softmax(logit)

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                 labels=T))
# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)


In [36]:
sess = tf.Session()

num_of_epoch = 2000

# Accuracy    
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

# K-Fold Cross Validation
cv = 5          # Fold의 수
results = []    
                
kf = KFold(n_splits=cv, shuffle=True) 

# logits and labels must be broadcastable: logits_size=[432000,10] labels_size=[48000,10]
# 이런 오류가 생김 Flatten이랑 validation에 문제가 있는듯

for training_idx, validation_idx in kf.split(x_data_train_norm):
    training_x = x_data_train_norm[training_idx] # Fancy indexing
    training_t = t_data_train_onehot[training_idx]
    
    val_x = x_data_train_norm[validation_idx]
    val_t = t_data_train_onehot[validation_idx]
    
    # run_train(sess,training_x,training_t)
    print('### 학습 시작 ###')
    sess.run(tf.global_variables_initializer())
    
    for step in range(num_of_epoch):
        _, loss_val = sess.run([train,loss],
                                   feed_dict={X: training_x, T: training_t})
    
        if step % 100 == 0:
            print('Loss : {}'.format(loss_val))
    
    acc = sess.run(accuracy, feed_dict={X:val_x, T:val_t})
    print('측정한 각각의 결과값 : {}'.format(acc))
    results.append(acc)


print('최종 K-Fold 교차검증을 사용한 Accuracy : {}'.format(np.mean(results)))

### 학습 시작 ###


InvalidArgumentError: logits and labels must be broadcastable: logits_size=[432000,10] labels_size=[48000,10]
	 [[node softmax_cross_entropy_with_logits_4 (defined at C:\Users\s_csmscox\Anaconda3\envs\tf1\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]

Original stack trace for 'softmax_cross_entropy_with_logits_4':
  File "C:\Users\s_csmscox\Anaconda3\envs\tf1\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\s_csmscox\Anaconda3\envs\tf1\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\s_csmscox\Anaconda3\envs\tf1\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\s_csmscox\Anaconda3\envs\tf1\lib\site-packages\traitlets\config\application.py", line 845, in launch_instance
    app.start()
  File "C:\Users\s_csmscox\Anaconda3\envs\tf1\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
    self.io_loop.start()
  File "C:\Users\

In [ ]:
result = sess.run(H, feed_dict={X: x_data_test_norm})
pred = []

for i in result:
    m = i.max()
    for j in range(10):
        if i[j] == m:
            pred.append(j)
            break

cnt = 0
for predict, answer in zip(pred, test_label):
    if predict == answer:
        cnt += 1

print("Accuracy : {}".format((cnt/len(pred))))